# Import the required libraries

In [1]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from typing import Optional, Dict
from toolbox_core import ToolboxSyncClient

# Define the model configuration

In [ ]:
MODEL = "openrouter/z-ai/glm-4.5-air:free"
llm = LiteLlm(model=MODEL)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", "content": "Hello"}], 
                                tools=[]))

print("\nOpenRouter is ready for use.")

ModelResponse(id='gen-1762062123-Fidsex2Fle0ZV8difSs3', created=1762062124, model='z-ai/glm-4.5-air:free', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="\nHello! 👋 How can I assist you today? Feel free to ask any question or share what's on your mind!", role='assistant', tool_calls=None, function_call=None, reasoning_content='Okay, the user just said "Hello". That\'s a simple greeting. I need to respond warmly and invitingly to make them feel comfortable. \n\nHmm, since they didn\'t provide any specific context or question, they might be testing the waters or starting a casual conversation. Maybe they\'re unsure how to begin. I should keep it open-ended but friendly. \n\nI recall that in my guidelines, I\'m supposed to be concise but helpful. Adding an emoji could make it feel more approachable. The smiley face seems right—it\'s universal and not too formal. \n\nThe phrase "How can I assist you today?" covers

# Load the tools from the MCP Toolbox

In [3]:
client = ToolboxSyncClient("http://127.0.0.1:5000") 

# This will load all tools
all_tools = client.load_toolset()

print(f"Loaded {all_tools} tools from MCP server")

Loaded [<toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020984C92060>, <toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020985F62CC0>, <toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020984BF27E0>, <toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020985FA1040>, <toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020985FA0F20>, <toolbox_core.sync_tool.ToolboxSyncTool object at 0x0000020985FA0F50>] tools from MCP server


# Create the runner

In [4]:
# Global cache for runners (maintains sessions)
_runner_cache: Dict[str, tuple] = {}

async def chat_with_agent(
    agent: Agent,
    message: str,
    user_id: str = "user_001",
    session_id: Optional[str] = None,
    maintain_session: bool = True,
    verbose: bool = True
) -> str:
    """
    Send a message to an agent and get the response.
    """
    if session_id is None:
        session_id = f"{agent.name}_session"
    
    app_name = f"{agent.name}"
    cache_key = f"{agent.name}_{user_id}_{session_id}"
    
    # Reuse or create runner
    if maintain_session and cache_key in _runner_cache:
        runner = _runner_cache[cache_key]
    else:
        session_service = InMemorySessionService()
        await session_service.create_session(
            app_name=app_name,
            user_id=user_id,
            session_id=session_id
        )
        
        runner = Runner(
            agent=agent,
            app_name=app_name,
            session_service=session_service
        )
        
        if maintain_session:
            _runner_cache[cache_key] = runner
    
    if verbose:
        print(f"\n>>> User Message: {message}")
    
    content = types.Content(role='user', parts=[types.Part(text=message)])
    final_response_text = None
    
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        if verbose:
            print(f"[Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}")
        
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            break

    if verbose:
        print(f"<<< Agent Response: {final_response_text}")
    
    return final_response_text

# Create the agent

In [5]:
db_agent = Agent(
    name="agents",
    model=llm,
    description="An intelligent database assistant for HDB resale transaction data",
    instruction="""You are a helpful database assistant with access to HDB resale transaction data from 2017 onwards.

**Your Role:**
- Answer user questions using the available tools
- Always respond in clear, natural language
- Format prices with proper currency notation (e.g., $500,000)
- Format large numbers with commas for readability

**Response Guidelines:**
- After calling any tool, ALWAYS provide a complete natural language summary
- Never output raw JSON or technical data structures
- Be conversational and helpful
- If a query returns no results, explain this clearly

**Available Data:**
The database contains HDB resale transactions with information about prices, locations (towns), flat types, flat models, floor areas, lease dates, and transaction dates.

Use the available tools intelligently to answer user questions. Each tool's description explains when to use it.""",
    tools=all_tools,
)

print(f"\nAgent '{db_agent.name}' created successfully with {len(all_tools)} tools")


Agent 'agents' created successfully with 6 tools


# Chat with the agent

In [6]:
response_avg_tampines = await chat_with_agent(
    db_agent, 
    "What is the average resale price for 4 ROOM flats in Tampines?",
    user_id="user_001",
    maintain_session=True
)


>>> User Message: What is the average resale price for 4 ROOM flats in Tampines?
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: True
<<< Agent Response: 

The average resale price for 4 ROOM flats in Tampines is **$371,135**.

This represents the average price across all 4 ROOM HDB resale transactions recorded in Tampines since 2017.


In [7]:
response_persistence_test = await chat_with_agent(
    db_agent, 
    "Using the average price from the last step, how many 5 ROOM flats in Tampines have a resale price greater than that value?",
    user_id="user_001", 
    maintain_session=True
)


>>> User Message: Using the average price from the last step, how many 5 ROOM flats in Tampines have a resale price greater than that value?
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: True
<<< Agent Response: 

Based on the data, there are **0** 5 ROOM flats in Tampines with a resale price greater than the average 4 ROOM flat price of $371,135.

This indicates that 5 ROOM flats in Tampines generally have prices below the average price of 4 ROOM flats in the same area, which is an interesting finding that might suggest the specific market conditions or types of 5 ROOM properties available in Tampines.Based on the average price of $371,135 for 4 ROOM flats in Tampines, there are **0** 5 ROOM flats in Tampines with resale prices greater than that amount.

This suggests that 5 ROOM flats in Tampines generally sell for less than the average price of 4 ROOM flats in the same area, which is a

In [8]:
response_switch = await chat_with_agent(
    db_agent, 
    "What is the median price in Jurong West?",
    user_id="user_001", 
    maintain_session=True
)


>>> User Message: What is the median price in Jurong West?
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: True
<<< Agent Response: 

The median price in Jurong West is **$325,000**. This represents the middle value of all resale transaction prices in the area, meaning half of the flats sold for less than this amount and half sold for more.


In [9]:
response_memory = await chat_with_agent(
    db_agent, 
    "What was the average price we calculated for 4 ROOM flats in Tampines?",
    user_id="user_001", 
    maintain_session=True
)


>>> User Message: What was the average price we calculated for 4 ROOM flats in Tampines?
[Event] Author: agents, Type: Event, Final: True
<<< Agent Response: 
The average price we calculated for 4 ROOM flats in Tampines was **$371,135**.


In [10]:
response_reset = await chat_with_agent(
  db_agent, 
  "What is the total count of flats in Tampines?", 
  user_id="user_002",
  session_id=None, 
  maintain_session=False
)


>>> User Message: What is the total count of flats in Tampines?
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: False
[Event] Author: agents, Type: Event, Final: True
<<< Agent Response: 
Based on the HDB resale transaction data from 2017 onwards, there are **51,077** flats in Tampines.
